# Preprocessing

In [2]:
import os
import hashlib

def get_file_hash(filepath):
    """Compute SHA-256 hash of a file."""
    hasher = hashlib.sha256()
    with open(filepath, "rb") as f:
        while chunk := f.read(8192):
            hasher.update(chunk)
    return hasher.hexdigest()

def find_duplicate_images(folder_path):
    """Find and print duplicate images in a folder and its subfolders."""
    hash_map = {}  # Stores hash -> file path
    duplicates = []

    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)

            # Skip non-image files (optional)
            if not file.lower().endswith(('.jpg')):
                continue

            file_hash = get_file_hash(file_path)

            if file_hash in hash_map:
                duplicates.append((file_path, hash_map[file_hash]))
            else:
                hash_map[file_hash] = file_path

    if duplicates:
        print("Duplicate Images Found:")
        for dup, original in duplicates:
            print(f"{dup} is a duplicate of {original}")
    else:
        print("All images are unique.")

folder = r"idd20kII/leftImg8bit/train/"  
find_duplicate_images(folder)


All images are unique.


In [3]:
import json
import os
import numpy as np
from PIL import Image

def polygon_area(polygon):
    """Calculate the area of a polygon using the Shoelace formula."""
    x = [p[0] for p in polygon]
    y = [p[1] for p in polygon]
    return 0.5 * abs(sum(x[i] * y[i+1] - x[i+1] * y[i] for i in range(len(x)-1)) + x[-1] * y[0] - x[0] * y[-1])

def calculate_bbox(polygon):
    """Calculate bounding box from polygon points [x1, y1, x2, y2, ...] format."""
    x_coords = [p[0] for p in polygon]
    y_coords = [p[1] for p in polygon]
    
    # COCO format is [x, y, width, height]
    x_min = min(x_coords)
    y_min = min(y_coords)
    width = max(x_coords) - x_min
    height = max(y_coords) - y_min
    
    return [x_min, y_min, width, height]

def convert_to_coco(input_files, output_file):
    """
    Convert custom JSON format to COCO format.
    
    Args:
        input_files: List of tuples containing (image_path, json_path)
        output_file: Path to save the COCO format JSON
    """
    # Initialize COCO format structure
    coco_data = {
        "info": {
            "description": "Converted from custom format",
            "url": "",
            "version": "1.0",
            "year": "",
            "contributor": "",
            "date_created": ""
        },
        "licenses": [
            {
                "id": 1,
                "name": "",
                "url": ""
            }
        ],
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    # Create a mapping of category names to ids
    category_map = {}
    category_id = 1
    
    # Keep track of image and annotation IDs
    image_id = 1
    annotation_id = 1
    
    # Process each image and its annotations
    for img_path, json_path in input_files:
        # Get image dimensions
        try:
            with Image.open(img_path) as img:
                width, height = img.size
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # Use dimensions from JSON if available
            with open(json_path, 'r') as f:
                json_data = json.load(f)
                width = json_data.get("imgWidth", 0)
                height = json_data.get("imgHeight", 0)
        
        # Add image to COCO format
        coco_data["images"].append({
            "id": image_id,
            "width": width,
            "height": height,
            "image_path": img_path,
            "license": 1,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": ""
        })
        
        # Process annotations
        try:
            with open(json_path, 'r') as f:
                json_data = json.load(f)
                
                for obj in json_data.get("objects", []):
                    if obj.get("deleted", 0) == 1:
                        continue  # Skip deleted objects
                    
                    label = obj.get("label", "unknown")
                    polygon = obj.get("polygon", [])
                    
                    # Skip incomplete polygons
                    if len(polygon) < 3:
                        continue
                    
                    # Add category if not seen before
                    if label not in category_map:
                        category_map[label] = category_id
                        coco_data["categories"].append({
                            "id": category_id,
                            "name": label,
                            "supercategory": "none"
                        })
                        category_id += 1
                    
                    # Calculate bounding box and area
                    bbox = calculate_bbox(polygon)
                    area = polygon_area(polygon)
                    
                    # Flatten the polygon for COCO format
                    flattened_polygon = [coord for point in polygon for coord in point]
                    
                    # Add annotation
                    coco_data["annotations"].append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": category_map[label],
                        "segmentation": [flattened_polygon],  # COCO expects a list of polygons
                        "area": area,
                        "bbox": bbox,
                        "iscrowd": 0,
                        "attributes": {
                            "date": obj.get("date", ""),
                            "verified": obj.get("verified", 0),
                            "user": obj.get("user", "")
                        }
                    })
                    
                    annotation_id += 1
        except Exception as e:
            print(f"Error processing annotations for {json_path}: {e}")
        
        image_id += 1
    
    # Write COCO format to file
    with open(output_file, 'w') as f:
        json.dump(coco_data, f, indent=2)
    
    print(f"Conversion complete. COCO format saved to {output_file}")
    print(f"Converted {image_id-1} images and {annotation_id-1} annotations")
    print(f"Found {len(coco_data['categories'])} categories: {', '.join(category_map.keys())}")

In [4]:
from glob import glob
import re
import numpy as np
def convert_path(p):
    # Replace the folder part
    
    p = re.sub(r'^idd20kII/leftImg8bit/train', 'idd20kII/gtFine/train', p)
    # Replace the filename part
    p = re.sub(r'_leftImg8bit\.jpg$', '_gtFine_polygons.json', p)
    return p

image = glob('idd20kII/leftImg8bit/train/*/*_leftImg8bit.jpg')
labels= [convert_path(p) for p in image]

print(f"Length of the dataset: {len(image)} and {len(labels)}" )
print(image[0], labels[0])

Length of the dataset: 7034 and 7034
idd20kII/leftImg8bit/train/368/frame1156_leftImg8bit.jpg idd20kII/gtFine/train/368/frame1156_gtFine_polygons.json


In [5]:
convert_to_coco(list(zip(image, labels)), 'idd20kII_train.json')

KeyboardInterrupt: 

In [6]:
convert_to_coco(list(zip(image, labels))[:100], 'idd20kII_train_subset.json')

Conversion complete. COCO format saved to idd20kII_train_subset.json
Converted 100 images and 9019 annotations
Found 33 categories: road, sky, non-drivable fallback, vegetation, obs-str-bar-fallback, fence, guard rail, curb, vehicle fallback, car, pole, traffic sign, fallback background, truck, sidewalk, building, drivable fallback, trailer, rider, motorcycle, person, billboard, autorickshaw, bus, wall, out of roi, animal, bicycle, traffic light, caravan, polegroup, bridge, rectification border


In [ ]:
from glob import glob
import re
import numpy as np
def convert_path(p):
    # Replace the folder part
    
    p = re.sub(r'^idd20kII/leftImg8bit/val', 'idd20kII/gtFine/val', p)
    # Replace the filename part
    p = re.sub(r'_leftImg8bit\.jpg$', '_gtFine_polygons.json', p)
    return p

image = glob('idd20kII/leftImg8bit/val/*/*_leftImg8bit.jpg')
labels= [convert_path(p) for p in image]

print(f"Length of the dataset: {len(image)} and {len(labels)}" )
print(image[0], labels[0])

Length of the dataset: 1055 and 1055
idd20kII/leftImg8bit/val/557/frame0149_leftImg8bit.jpg idd20kII/gtFine/val/557/frame0149_gtFine_polygons.json


In [ ]:
convert_to_coco(list(zip(image, labels)), 'idd20kII_val.json')

idd20kII/leftImg8bit/val/557/frame0149_leftImg8bit.jpg idd20kII/gtFine/val/557/frame0149_gtFine_polygons.json


Conversion complete. COCO format saved to idd20kII_coco_format.json
Converted 8089 images and 792146 annotations
Found 39 categories: road, sky, non-drivable fallback, vegetation, obs-str-bar-fallback, fence, guard rail, curb, vehicle fallback, car, pole, traffic sign, fallback background, truck, sidewalk, building, drivable fallback, trailer, rider, motorcycle, person, billboard, autorickshaw, bus, wall, out of roi, animal, bicycle, traffic light, caravan, polegroup, bridge, rectification border, train, parking, rail track, tunnel, unlabeled, ego vehicle


In [9]:
dataset= json.load(open('idd20kII_coco_format.json'))


In [10]:
dataset.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [11]:
dataset['images'][0].keys()

dict_keys(['id', 'width', 'height', 'image_path', 'license', 'flickr_url', 'coco_url', 'date_captured'])

In [ ]:
daraset